In [1]:
import requests
import json

url = "http://10.15.102.186:9000/api/chat"

# ====== 2. 请求体 ======
data = {
    "model": "qwen3:30b-a3b-instruct-2507-q8_0",
    "messages": [
        {
            "role": "user",
            "content": "Hello, how are you?"
        }
    ],
    "stream": False
}

# ====== 3. 发送请求 ======
try:
    response = requests.post(
        url,
        json=data,
        timeout=600  
    )
    response.raise_for_status()

    res_json = response.json()
    print("\n=== 模型响应 ===\n")
    print(json.dumps(res_json, indent=2, ensure_ascii=False))

except Exception as e:
    print("请求失败：", e)


=== 模型响应 ===

{
  "model": "qwen3:30b-a3b-instruct-2507-q8_0",
  "created_at": "2025-12-01T09:58:58.667524826Z",
  "message": {
    "role": "assistant",
    "content": "Hello! I'm functioning well, thank you for asking! I'm always excited to help with questions or chat about interesting topics. How can I assist you today? 😊"
  },
  "done_reason": "stop",
  "done": true,
  "total_duration": 10201212918,
  "load_duration": 9694608910,
  "prompt_eval_count": 14,
  "prompt_eval_duration": 173705211,
  "eval_count": 36,
  "eval_duration": 331855673
}


In [2]:
from pymilvus import connections, utility, Collection

connections.connect(
    alias="default",
    host="10.19.48.181",
    port="19530",
    user="cs286_2025_group8",     # 你的账号
    password="Group8",              # 你的密码
    db_name="cs286_2025_group8",    # ⭐ 必须指定你在 UI 中看到的数据库名
    secure=False,                   # 内网一般不启用 TLS
)


col = Collection("xfel_bibs_collection")
print(col.schema)


print("\n=== Fields ===")
fields = [f.name for f in col.schema.fields]
print(fields)

# 选择你想看的字段（向量先不展示）
output_fields = ["pk", "title", "year", "journal", "page", "text", "start_index"]

print("\n=== First 3 Rows ===")
res = col.query(
    expr="pk >=0 ",
    output_fields=output_fields,
    limit=3
)

for r in res:
    print("\n--- Row ---")
    for k, v in r.items():
        print(f"{k}: {v}")

{'auto_id': True, 'description': 'XFEL文献向量库 BGE-M3混合检索; size=2000; overlap=200', 'fields': [{'name': 'title', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000}}, {'name': 'doi', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000}}, {'name': 'year', 'description': '', 'type': <DataType.INT16: 3>}, {'name': 'journal', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000}}, {'name': 'page', 'description': '', 'type': <DataType.INT16: 3>}, {'name': 'start_index', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'dense_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}, {'name': 'sparse_vector', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 10

In [4]:
from langchain_classic.retrievers.document_compressors.cross_encoder_rerank import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder


model_name = "BAAI/bge-reranker-v2-m3"
rerank_model = HuggingFaceCrossEncoder(model_name=model_name)
compressor = CrossEncoderReranker(model=rerank_model, top_n=3)






Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


KeyboardInterrupt: 